<a href="https://colab.research.google.com/github/piabona/Dacon-Kaggle/blob/main/JEJU/%EC%A0%9C%EC%A3%BC_%ED%8A%B9%EC%82%B0%EB%AC%BC_%ED%8C%80%EC%B5%9C%EC%A2%85_(667_01865).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install autogluon

In [ ]:
import pandas as pd
import numpy as np
import datetime
import os
import torch
import random

DATA_PATH = "/content/drive/MyDrive/DACON/제주도/data/"
SEED = 42

In [ ]:
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.enabled = False


seed_everything(SEED)

In [ ]:
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

In [ ]:
train_df = pd.read_csv(f"{DATA_PATH}train.csv")
trade_df = pd.read_csv(f"{DATA_PATH}international_trade.csv")
test_df = pd.read_csv(f"{DATA_PATH}test.csv")

In [ ]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
train_df['year'] = train_df['timestamp'].dt.year
train_df['month'] = train_df['timestamp'].dt.month
train_df['week'] = train_df['timestamp'].dt.isocalendar().week.astype(np.int32)
train_df['day'] = train_df['timestamp'].dt.day
train_df['day_of_week'] = train_df['timestamp'].dt.day_of_week
train_df['sin_month'] = np.sin(2 * np.pi * train_df['month']/12)
train_df['cos_month'] = np.cos(2 * np.pi * train_df['month']/12)
train_df['sin_week'] = np.sin(2 * np.pi * train_df['week']/52)
train_df['cos_week'] = np.cos(2 * np.pi * train_df['week']/52)
train_df['sin_dayofweek'] = np.sin(2 * np.pi * (train_df['day_of_week'] + 1) / 7)
train_df['cos_dayofweek'] = np.cos(2 * np.pi * (train_df['day_of_week'] + 1) / 7)
train_df

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,week,day,day_of_week,sin_month,cos_month,sin_week,cos_week,sin_dayofweek,cos_dayofweek
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,0.500000,8.660254e-01,0.120537,0.992709,0.974928,-0.222521
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,2,2,0.500000,8.660254e-01,0.120537,0.992709,0.433884,-0.900969
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,3,3,0.500000,8.660254e-01,0.120537,0.992709,-0.433884,-0.900969
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,4,4,0.500000,8.660254e-01,0.120537,0.992709,-0.974928,-0.222521
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,5,5,0.500000,8.660254e-01,0.120537,0.992709,-0.781831,0.623490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,9,27,0,0.866025,5.000000e-01,0.885456,0.464723,0.781831,0.623490
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,9,28,1,0.866025,5.000000e-01,0.885456,0.464723,0.974928,-0.222521
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,9,1,2,1.000000,6.123234e-17,0.885456,0.464723,0.433884,-0.900969
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,9,2,3,1.000000,6.123234e-17,0.885456,0.464723,-0.433884,-0.900969


In [ ]:
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
test_df['year'] = test_df['timestamp'].dt.year
test_df['month'] = test_df['timestamp'].dt.month
test_df['week'] = test_df['timestamp'].dt.isocalendar().week.astype(np.int32)
test_df['day'] = test_df['timestamp'].dt.day
test_df['day_of_week'] = test_df['timestamp'].dt.day_of_week
test_df['sin_month'] = np.sin(2 * np.pi * test_df['month']/12.0)
test_df['cos_month'] = np.cos(2 * np.pi * test_df['month']/12.0)
test_df['sin_week'] = np.sin(2 * np.pi * test_df['week'] / 52)
test_df['cos_week'] = np.cos(2 * np.pi * test_df['week'] / 52)
test_df['sin_dayofweek'] = np.sin(2 * np.pi * (test_df['day_of_week'] + 1) / 7)
test_df['cos_dayofweek'] = np.cos(2 * np.pi * (test_df['day_of_week'] + 1) / 7)
test_df

,ID,timestamp,item,corporation,location,year,month,week,day,day_of_week,sin_month,cos_month,sin_week,cos_week,sin_dayofweek,cos_dayofweek
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,9,4,5,1.0,6.123234e-17,0.885456,4.647232e-01,-7.818315e-01,0.623490
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,9,5,6,1.0,6.123234e-17,0.885456,4.647232e-01,-2.449294e-16,1.000000
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,10,6,0,1.0,6.123234e-17,0.935016,3.546049e-01,7.818315e-01,0.623490
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,10,7,1,1.0,6.123234e-17,0.935016,3.546049e-01,9.749279e-01,-0.222521
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,10,8,2,1.0,6.123234e-17,0.935016,3.546049e-01,4.338837e-01,-0.900969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J,2023,3,13,27,0,1.0,6.123234e-17,1.000000,-1.608123e-16,7.818315e-01,0.623490
1088,RD_F_J_20230328,2023-03-28,RD,F,J,2023,3,13,28,1,1.0,6.123234e-17,1.000000,-1.608123e-16,9.749279e-01,-0.222521
1089,RD_F_J_20230329,2023-03-29,RD,F,J,2023,3,13,29,2,1.0,6.123234e-17,1.000000,-1.608123e-16,4.338837e-01,-0.900969
1090,RD_F_J_20230330,2023-03-30,RD,F,J,2023,3,13,30,3,1.0,6.123234e-17,1.000000,-1.608123e-16,-4.338837e-01,-0.900969


In [ ]:
# 주 수확 시기 1, 2, 10, 11, 12

train_df['harvest'] = np.where((train_df['month'] <= 2) | (train_df['month'] >= 10), 0, 1)
test_df['harvest'] = 1

train_df

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,week,day,day_of_week,sin_month,cos_month,sin_week,cos_week,sin_dayofweek,cos_dayofweek,harvest
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,0.500000,8.660254e-01,0.120537,0.992709,0.974928,-0.222521,0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,2,2,0.500000,8.660254e-01,0.120537,0.992709,0.433884,-0.900969,0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,3,3,0.500000,8.660254e-01,0.120537,0.992709,-0.433884,-0.900969,0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,4,4,0.500000,8.660254e-01,0.120537,0.992709,-0.974928,-0.222521,0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,5,5,0.500000,8.660254e-01,0.120537,0.992709,-0.781831,0.623490,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,9,27,0,0.866025,5.000000e-01,0.885456,0.464723,0.781831,0.623490,0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,9,28,1,0.866025,5.000000e-01,0.885456,0.464723,0.974928,-0.222521,0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,9,1,2,1.000000,6.123234e-17,0.885456,0.464723,0.433884,-0.900969,1
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,9,2,3,1.000000,6.123234e-17,0.885456,0.464723,-0.433884,-0.900969,1


In [ ]:
# 일요일 여부

train_df['sunday'] = np.where(train_df['day_of_week'] == 6, 1, 0)
test_df['sunday'] = np.where(test_df['day_of_week'] == 6, 1, 0)

train_df

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,week,day,day_of_week,sin_month,cos_month,sin_week,cos_week,sin_dayofweek,cos_dayofweek,harvest,sunday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,0.500000,8.660254e-01,0.120537,0.992709,0.974928,-0.222521,0,0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,2,2,0.500000,8.660254e-01,0.120537,0.992709,0.433884,-0.900969,0,0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,3,3,0.500000,8.660254e-01,0.120537,0.992709,-0.433884,-0.900969,0,0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,4,4,0.500000,8.660254e-01,0.120537,0.992709,-0.974928,-0.222521,0,0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,5,5,0.500000,8.660254e-01,0.120537,0.992709,-0.781831,0.623490,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,9,27,0,0.866025,5.000000e-01,0.885456,0.464723,0.781831,0.623490,0,0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,9,28,1,0.866025,5.000000e-01,0.885456,0.464723,0.974928,-0.222521,0,0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,9,1,2,1.000000,6.123234e-17,0.885456,0.464723,0.433884,-0.900969,1,0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,9,2,3,1.000000,6.123234e-17,0.885456,0.464723,-0.433884,-0.900969,1,0


In [ ]:
train_df.rename(columns={'supply(kg)' : 'supply', 'price(원/kg)' : 'price'}, inplace=True)
train_df

,ID,timestamp,item,corporation,location,supply,price,year,month,week,day,day_of_week,sin_month,cos_month,sin_week,cos_week,sin_dayofweek,cos_dayofweek,harvest,sunday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1,0.500000,8.660254e-01,0.120537,0.992709,0.974928,-0.222521,0,0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,2,2,0.500000,8.660254e-01,0.120537,0.992709,0.433884,-0.900969,0,0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,3,3,0.500000,8.660254e-01,0.120537,0.992709,-0.433884,-0.900969,0,0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,4,4,0.500000,8.660254e-01,0.120537,0.992709,-0.974928,-0.222521,0,0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,5,5,0.500000,8.660254e-01,0.120537,0.992709,-0.781831,0.623490,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,9,27,0,0.866025,5.000000e-01,0.885456,0.464723,0.781831,0.623490,0,0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,9,28,1,0.866025,5.000000e-01,0.885456,0.464723,0.974928,-0.222521,0,0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,9,1,2,1.000000,6.123234e-17,0.885456,0.464723,0.433884,-0.900969,1,0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,9,2,3,1.000000,6.123234e-17,0.885456,0.464723,-0.433884,-0.900969,1,0


In [ ]:
month_dayofweek_mean = train_df.groupby(['item', 'corporation', 'location', 'month', 'day_of_week'])['price'].mean().reset_index()
month_dayofweek_mean = month_dayofweek_mean.rename(columns={'price' : 'month_dayofweek_mean'})
train_df = train_df.merge(month_dayofweek_mean, on=['item', 'corporation', 'location', 'month', 'day_of_week'], how='left')
test_df = test_df.merge(month_dayofweek_mean, on=['item', 'corporation', 'location', 'month', 'day_of_week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,day_of_week,sin_month,cos_month,sin_week,cos_week,sin_dayofweek,cos_dayofweek,harvest,sunday,month_dayofweek_mean
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,1,0.5,0.866025,0.120537,0.992709,0.974928,-0.222521,0,0,1560.545455
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,2,0.5,0.866025,0.120537,0.992709,0.433884,-0.900969,0,0,1716.818182
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,3,0.5,0.866025,0.120537,0.992709,-0.433884,-0.900969,0,0,1718.000000
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,4,0.5,0.866025,0.120537,0.992709,-0.974928,-0.222521,0,0,1771.318182
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,5,0.5,0.866025,0.120537,0.992709,-0.781831,0.623490,0,0,1498.863636


In [ ]:
month_dayofweek_std = train_df.groupby(['item', 'corporation', 'location', 'month', 'day_of_week'])['price'].std().reset_index()
month_dayofweek_std = month_dayofweek_std.rename(columns={'price' : 'month_dayofweek_std'})
train_df = train_df.merge(month_dayofweek_std, on=['item', 'corporation', 'location', 'month', 'day_of_week'], how='left')
test_df = test_df.merge(month_dayofweek_std, on=['item', 'corporation', 'location', 'month', 'day_of_week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,sin_month,cos_month,sin_week,cos_week,sin_dayofweek,cos_dayofweek,harvest,sunday,month_dayofweek_mean,month_dayofweek_std
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,0.5,0.866025,0.120537,0.992709,0.974928,-0.222521,0,0,1560.545455,624.919326
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,0.5,0.866025,0.120537,0.992709,0.433884,-0.900969,0,0,1716.818182,1145.872994
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,0.5,0.866025,0.120537,0.992709,-0.433884,-0.900969,0,0,1718.000000,676.837745
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,0.5,0.866025,0.120537,0.992709,-0.974928,-0.222521,0,0,1771.318182,1068.926136
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,0.5,0.866025,0.120537,0.992709,-0.781831,0.623490,0,0,1498.863636,780.810464


In [ ]:
month_dayofweek_max = train_df.groupby(['item', 'corporation', 'location', 'month', 'day_of_week'])['price'].max().reset_index()
month_dayofweek_max = month_dayofweek_max.rename(columns={'price' : 'month_dayofweek_max'})
train_df = train_df.merge(month_dayofweek_max, on=['item', 'corporation', 'location', 'month', 'day_of_week'], how='left')
test_df = test_df.merge(month_dayofweek_max, on=['item', 'corporation', 'location', 'month', 'day_of_week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,cos_month,sin_week,cos_week,sin_dayofweek,cos_dayofweek,harvest,sunday,month_dayofweek_mean,month_dayofweek_std,month_dayofweek_max
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,0.866025,0.120537,0.992709,0.974928,-0.222521,0,0,1560.545455,624.919326,2472.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,0.866025,0.120537,0.992709,0.433884,-0.900969,0,0,1716.818182,1145.872994,5054.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,0.866025,0.120537,0.992709,-0.433884,-0.900969,0,0,1718.000000,676.837745,2919.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,0.866025,0.120537,0.992709,-0.974928,-0.222521,0,0,1771.318182,1068.926136,5685.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,0.866025,0.120537,0.992709,-0.781831,0.623490,0,0,1498.863636,780.810464,2844.0


In [ ]:
week_mean = train_df.groupby(['item', 'corporation', 'location', 'week'])['price'].mean().reset_index()
week_mean = week_mean.rename(columns={'price' : 'week_mean'})
train_df = train_df.merge(week_mean, on=['item','corporation', 'location', 'week'], how='left')
test_df = test_df.merge(week_mean, on=['item','corporation', 'location', 'week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,sin_week,cos_week,sin_dayofweek,cos_dayofweek,harvest,sunday,month_dayofweek_mean,month_dayofweek_std,month_dayofweek_max,week_mean
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,0.120537,0.992709,0.974928,-0.222521,0,0,1560.545455,624.919326,2472.0,1099.735294
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,0.120537,0.992709,0.433884,-0.900969,0,0,1716.818182,1145.872994,5054.0,1099.735294
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,0.120537,0.992709,-0.433884,-0.900969,0,0,1718.000000,676.837745,2919.0,1099.735294
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,0.120537,0.992709,-0.974928,-0.222521,0,0,1771.318182,1068.926136,5685.0,1099.735294
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,0.120537,0.992709,-0.781831,0.623490,0,0,1498.863636,780.810464,2844.0,1099.735294


In [ ]:
week_std = train_df.groupby(['item', 'corporation', 'location', 'week'])['price'].std().reset_index()
week_std = week_std.rename(columns={'price' : 'week_std'})
train_df = train_df.merge(week_std, on=['item','corporation', 'location', 'week'], how='left')
test_df = test_df.merge(week_std, on=['item','corporation', 'location', 'week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,cos_week,sin_dayofweek,cos_dayofweek,harvest,sunday,month_dayofweek_mean,month_dayofweek_std,month_dayofweek_max,week_mean,week_std
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,0.992709,0.974928,-0.222521,0,0,1560.545455,624.919326,2472.0,1099.735294,852.06065
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,0.992709,0.433884,-0.900969,0,0,1716.818182,1145.872994,5054.0,1099.735294,852.06065
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,0.992709,-0.433884,-0.900969,0,0,1718.000000,676.837745,2919.0,1099.735294,852.06065
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,0.992709,-0.974928,-0.222521,0,0,1771.318182,1068.926136,5685.0,1099.735294,852.06065
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,0.992709,-0.781831,0.623490,0,0,1498.863636,780.810464,2844.0,1099.735294,852.06065


In [ ]:
month_mean = train_df.groupby(['item', 'corporation', 'location', 'month'])['price'].mean().reset_index()
month_mean = month_mean.rename(columns={'price' : 'month_mean'})
train_df = train_df.merge(month_mean, on=['item','corporation', 'location', 'month'], how='left')
test_df = test_df.merge(month_mean, on=['item','corporation', 'location', 'month'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,sin_dayofweek,cos_dayofweek,harvest,sunday,month_dayofweek_mean,month_dayofweek_std,month_dayofweek_max,week_mean,week_std,month_mean
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,0.974928,-0.222521,0,0,1560.545455,624.919326,2472.0,1099.735294,852.06065,1403.858065
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,0.433884,-0.900969,0,0,1716.818182,1145.872994,5054.0,1099.735294,852.06065,1403.858065
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,-0.433884,-0.900969,0,0,1718.000000,676.837745,2919.0,1099.735294,852.06065,1403.858065
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,-0.974928,-0.222521,0,0,1771.318182,1068.926136,5685.0,1099.735294,852.06065,1403.858065
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,-0.781831,0.623490,0,0,1498.863636,780.810464,2844.0,1099.735294,852.06065,1403.858065


In [ ]:
month_std = train_df.groupby(['item', 'corporation', 'location', 'month'])['price'].std().reset_index()
month_std = month_std.rename(columns={'price' : 'month_std'})
train_df = train_df.merge(month_std, on=['item','corporation', 'location', 'month'], how='left')
test_df = test_df.merge(month_std, on=['item','corporation', 'location', 'month'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,cos_dayofweek,harvest,sunday,month_dayofweek_mean,month_dayofweek_std,month_dayofweek_max,week_mean,week_std,month_mean,month_std
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,-0.222521,0,0,1560.545455,624.919326,2472.0,1099.735294,852.06065,1403.858065,988.542903
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,-0.900969,0,0,1716.818182,1145.872994,5054.0,1099.735294,852.06065,1403.858065,988.542903
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,-0.900969,0,0,1718.000000,676.837745,2919.0,1099.735294,852.06065,1403.858065,988.542903
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,-0.222521,0,0,1771.318182,1068.926136,5685.0,1099.735294,852.06065,1403.858065,988.542903
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,0.623490,0,0,1498.863636,780.810464,2844.0,1099.735294,852.06065,1403.858065,988.542903


In [ ]:
month_mean_supply = train_df.groupby(['item', 'corporation', 'location', 'month'])['supply'].mean().reset_index()
month_mean_supply = month_mean_supply.rename(columns={'supply' : 'month_mean_supply'})
train_df = train_df.merge(month_mean_supply, on=['item','corporation', 'location', 'month'], how='left')
test_df = test_df.merge(month_mean_supply, on=['item','corporation', 'location', 'month'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,harvest,sunday,month_dayofweek_mean,month_dayofweek_std,month_dayofweek_max,week_mean,week_std,month_mean,month_std,month_mean_supply
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,0,0,1560.545455,624.919326,2472.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,0,0,1716.818182,1145.872994,5054.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,0,0,1718.000000,676.837745,2919.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,0,0,1771.318182,1068.926136,5685.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,0,0,1498.863636,780.810464,2844.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839


In [ ]:
month_std_supply = train_df.groupby(['item', 'corporation', 'location', 'month'])['supply'].std().reset_index()
month_std_supply = month_std_supply.rename(columns={'supply' : 'month_std_supply'})
train_df = train_df.merge(month_std_supply, on=['item','corporation', 'location', 'month'], how='left')
test_df = test_df.merge(month_std_supply, on=['item','corporation', 'location', 'month'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,sunday,month_dayofweek_mean,month_dayofweek_std,month_dayofweek_max,week_mean,week_std,month_mean,month_std,month_mean_supply,month_std_supply
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,0,1560.545455,624.919326,2472.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,0,1716.818182,1145.872994,5054.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,0,1718.000000,676.837745,2919.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,0,1771.318182,1068.926136,5685.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,0,1498.863636,780.810464,2844.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794


In [ ]:
week_mean_supply = train_df.groupby(['item', 'corporation', 'location', 'week'])['supply'].mean().reset_index()
week_mean_supply = week_mean_supply.rename(columns={'supply' : 'week_mean_supply'})
train_df = train_df.merge(week_mean_supply, on=['item','corporation', 'location', 'week'], how='left')
test_df = test_df.merge(week_mean_supply, on=['item','corporation', 'location', 'week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,month_dayofweek_mean,month_dayofweek_std,month_dayofweek_max,week_mean,week_std,month_mean,month_std,month_mean_supply,month_std_supply,week_mean_supply
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,1560.545455,624.919326,2472.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,1716.818182,1145.872994,5054.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,1718.000000,676.837745,2919.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,1771.318182,1068.926136,5685.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,1498.863636,780.810464,2844.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176


In [ ]:
week_std_supply = train_df.groupby(['item', 'corporation', 'location', 'week'])['supply'].std().reset_index()
week_std_supply = week_std_supply.rename(columns={'supply' : 'week_std_supply'})
train_df = train_df.merge(week_std_supply, on=['item','corporation', 'location', 'week'], how='left')
test_df = test_df.merge(week_std_supply, on=['item','corporation', 'location', 'week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,month_dayofweek_std,month_dayofweek_max,week_mean,week_std,month_mean,month_std,month_mean_supply,month_std_supply,week_mean_supply,week_std_supply
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,624.919326,2472.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,1145.872994,5054.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,676.837745,2919.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,1068.926136,5685.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,780.810464,2844.0,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469


In [ ]:
train_df['total'] = train_df['supply'] * train_df['price']

week_mean_total = train_df.groupby(['item', 'corporation', 'location', 'week'])['total'].mean().reset_index()
week_mean_total = week_mean_total.rename(columns={'total' : 'week_mean_total'})
train_df = train_df.merge(week_mean_total, on=['item','corporation', 'location', 'week'], how='left')
test_df = test_df.merge(week_mean_total, on=['item','corporation', 'location', 'week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,week_mean,week_std,month_mean,month_std,month_mean_supply,month_std_supply,week_mean_supply,week_std_supply,total,week_mean_total
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,0.0,4.999182e+07
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,0.0,4.999182e+07
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,104718528.0,4.999182e+07
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,35200000.0,4.999182e+07
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,1099.735294,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,40440000.0,4.999182e+07


In [ ]:
week_std_total = train_df.groupby(['item', 'corporation', 'location', 'week'])['total'].std().reset_index()
week_std_total = week_std_total.rename(columns={'total' : 'week_std_total'})
train_df = train_df.merge(week_std_total, on=['item','corporation', 'location', 'week'], how='left')
test_df = test_df.merge(week_std_total, on=['item','corporation', 'location', 'week'], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,week_std,month_mean,month_std,month_mean_supply,month_std_supply,week_mean_supply,week_std_supply,total,week_mean_total,week_std_total
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,0.0,4.999182e+07,5.700237e+07
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,0.0,4.999182e+07,5.700237e+07
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,104718528.0,4.999182e+07,5.700237e+07
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,35200000.0,4.999182e+07,5.700237e+07
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,852.06065,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,40440000.0,4.999182e+07,5.700237e+07


In [ ]:
month_mean_total = train_df.groupby(['item', 'corporation', 'location', 'month'])['total'].mean().reset_index()
month_mean_total = month_mean_total.rename(columns={'total' : 'month_mean_total'})
train_df = train_df.merge(month_mean_total, on=['item','corporation', 'location', 'month'], how='left')
test_df = test_df.merge(month_mean_total, on=['item','corporation', 'location', 'month' ], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,month_mean,month_std,month_mean_supply,month_std_supply,week_mean_supply,week_std_supply,total,week_mean_total,week_std_total,month_mean_total
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,0.0,4.999182e+07,5.700237e+07,4.952021e+07
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,0.0,4.999182e+07,5.700237e+07,4.952021e+07
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,104718528.0,4.999182e+07,5.700237e+07,4.952021e+07
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,35200000.0,4.999182e+07,5.700237e+07,4.952021e+07
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,1403.858065,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,40440000.0,4.999182e+07,5.700237e+07,4.952021e+07


In [ ]:
month_std_total = train_df.groupby(['item', 'corporation', 'location', 'month'])['total'].std().reset_index()
month_std_total = month_std_total.rename(columns={'total' : 'month_std_total'})
train_df = train_df.merge(month_std_total, on=['item','corporation', 'location', 'month'], how='left')
test_df = test_df.merge(month_std_total, on=['item','corporation', 'location', 'month' ], how='left')

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,month_std,month_mean_supply,month_std_supply,week_mean_supply,week_std_supply,total,week_mean_total,week_std_total,month_mean_total,month_std_total
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,0.0,4.999182e+07,5.700237e+07,4.952021e+07,5.091995e+07
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,0.0,4.999182e+07,5.700237e+07,4.952021e+07,5.091995e+07
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,104718528.0,4.999182e+07,5.700237e+07,4.952021e+07,5.091995e+07
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,35200000.0,4.999182e+07,5.700237e+07,4.952021e+07,5.091995e+07
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,988.542903,29435.214839,31616.201794,33714.341176,40264.234469,40440000.0,4.999182e+07,5.700237e+07,4.952021e+07,5.091995e+07


In [ ]:
trade_df = trade_df[trade_df.품목명.str.contains('감귤|브로콜리|무|당근|양배추')]
trade_df = trade_df[(trade_df.품목명 != '무화과') & (trade_df.품목명 != '방울다다기 양배추')].reset_index(drop=True)
trade_df

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
0,2019-01,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562
1,2019-01,양배추,184650,94,395802,90,4
2,2019-01,당근,23150,22,7466150,2955,-2934
3,2019-01,감귤,58368,172,0,0,172
4,2019-02,꽃양배추와 브로콜리(broccoli),780,1,396870,399,-398
...,...,...,...,...,...,...,...
201,2023-02,꽃양배추와 브로콜리(broccoli),24,0,332640,352,-352
202,2023-02,양배추,13188,13,377456,104,-91
203,2023-02,당근,22510,20,9260020,3758,-3737
204,2023-02,순무,4000,4,2,0,4


In [ ]:
trade_df['year'] = trade_df['기간'].apply(lambda x : int(x[0:4]))
trade_df['month'] = trade_df['기간'].apply(lambda x : int(x[5:7]))
trade_df.drop(columns = '기간', inplace=True)
trade_df

,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month
0,꽃양배추와 브로콜리(broccoli),160,1,638913,563,-562,2019,1
1,양배추,184650,94,395802,90,4,2019,1
2,당근,23150,22,7466150,2955,-2934,2019,1
3,감귤,58368,172,0,0,172,2019,1
4,꽃양배추와 브로콜리(broccoli),780,1,396870,399,-398,2019,2
...,...,...,...,...,...,...,...,...
201,꽃양배추와 브로콜리(broccoli),24,0,332640,352,-352,2023,2
202,양배추,13188,13,377456,104,-91,2023,2
203,당근,22510,20,9260020,3758,-3737,2023,2
204,순무,4000,4,2,0,4,2023,2


In [ ]:
trade_df['품목명'].unique()

array(['꽃양배추와 브로콜리(broccoli)', '양배추', '당근', '감귤', '순무'], dtype=object)

In [ ]:
train_df['item'].unique()

array(['TG', 'CR', 'CB', 'RD', 'BC'], dtype=object)

In [ ]:
lst = ['BC', 'CB', 'CR', 'TG', 'RD']
for i, item in enumerate(trade_df['품목명'].unique()):
    trade_df.loc[trade_df['품목명'] == item, '품목명'] = lst[i]

trade_df

,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month
0,BC,160,1,638913,563,-562,2019,1
1,CB,184650,94,395802,90,4,2019,1
2,CR,23150,22,7466150,2955,-2934,2019,1
3,TG,58368,172,0,0,172,2019,1
4,BC,780,1,396870,399,-398,2019,2
...,...,...,...,...,...,...,...,...
201,BC,24,0,332640,352,-352,2023,2
202,CB,13188,13,377456,104,-91,2023,2
203,CR,22510,20,9260020,3758,-3737,2023,2
204,RD,4000,4,2,0,4,2023,2


In [ ]:
trade_df = trade_df.rename(columns={
    '품목명': 'item',
    '수출 중량': 'out_weight',
    '수출 금액': 'out_value',
    '수입 중량': 'in_weight',
    '수입 금액': 'in_value',
    '무역수지': 'tradebalance'
})

trade_df.head()

,item,out_weight,out_value,in_weight,in_value,tradebalance,year,month
0,BC,160,1,638913,563,-562,2019,1
1,CB,184650,94,395802,90,4,2019,1
2,CR,23150,22,7466150,2955,-2934,2019,1
3,TG,58368,172,0,0,172,2019,1
4,BC,780,1,396870,399,-398,2019,2


In [ ]:
trade_df['out_price'] = trade_df['out_value'] / trade_df['out_weight']
trade_df['in_price'] = trade_df['in_value'] / trade_df['in_weight']

trade_df

,item,out_weight,out_value,in_weight,in_value,tradebalance,year,month,out_price,in_price
0,BC,160,1,638913,563,-562,2019,1,0.006250,0.000881
1,CB,184650,94,395802,90,4,2019,1,0.000509,0.000227
2,CR,23150,22,7466150,2955,-2934,2019,1,0.000950,0.000396
3,TG,58368,172,0,0,172,2019,1,0.002947,NaN
4,BC,780,1,396870,399,-398,2019,2,0.001282,0.001005
...,...,...,...,...,...,...,...,...,...,...
201,BC,24,0,332640,352,-352,2023,2,0.000000,0.001058
202,CB,13188,13,377456,104,-91,2023,2,0.000986,0.000276
203,CR,22510,20,9260020,3758,-3737,2023,2,0.000888,0.000406
204,RD,4000,4,2,0,4,2023,2,0.001000,0.000000


In [ ]:
train_df = train_df.merge(trade_df, on=['year', 'month', 'item'], how='left')
test_df = test_df.merge(trade_df, on=['year', 'month', 'item'], how='left')

train_df

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,week_std_total,month_mean_total,month_std_total,out_weight,out_value,in_weight,in_value,tradebalance,out_price,in_price
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,5.700237e+07,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,0.002947,NaN
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,5.700237e+07,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,0.002947,NaN
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,5.700237e+07,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,0.002947,NaN
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,5.700237e+07,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,0.002947,NaN
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,5.700237e+07,4.952021e+07,5.091995e+07,58368.0,172.0,0.0,0.0,172.0,0.002947,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,9,...,7.366100e+07,1.476583e+08,9.325641e+07,4000.0,4.0,2.0,0.0,4.0,0.001000,0.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,9,...,7.366100e+07,1.476583e+08,9.325641e+07,4000.0,4.0,2.0,0.0,4.0,0.001000,0.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,9,...,7.366100e+07,1.469691e+08,7.611450e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,9,...,7.366100e+07,1.469691e+08,7.611450e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
trade_month_mean = trade_df.groupby(['item', 'month'])[['out_weight', 'out_value', 'in_weight', 'in_value', 'tradebalance']].mean().reset_index()
trade_month_mean = trade_month_mean.rename(columns={'out_weight' : 'out_weight_mean',
                                                    'out_value': 'out_value_mean',
                                                    'in_weight': 'in_weight_mean',
                                                    'in_value': 'in_value_mean',
                                                    'tradebalance' : 'tradebalance_mean'})
train_df = train_df.merge(trade_month_mean, on=['item', 'month'], how='left')
test_df = test_df.merge(trade_month_mean, on=['item', 'month' ], how='left')

train_df

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,in_weight,in_value,tradebalance,out_price,in_price,out_weight_mean,out_value_mean,in_weight_mean,in_value_mean,tradebalance_mean
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,0.0,0.0,172.0,0.002947,NaN,192621.6,317.4,0.2,0.0,317.4
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,0.0,0.0,172.0,0.002947,NaN,192621.6,317.4,0.2,0.0,317.4
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,0.0,0.0,172.0,0.002947,NaN,192621.6,317.4,0.2,0.0,317.4
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,0.0,0.0,172.0,0.002947,NaN,192621.6,317.4,0.2,0.0,317.4
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,0.0,0.0,172.0,0.002947,NaN,192621.6,317.4,0.2,0.0,317.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,9,...,2.0,0.0,4.0,0.001000,0.0,4000.0,4.0,2.0,0.0,4.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,9,...,2.0,0.0,4.0,0.001000,0.0,4000.0,4.0,2.0,0.0,4.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_df

,ID,timestamp,item,corporation,location,year,month,week,day,day_of_week,...,in_weight,in_value,tradebalance,out_price,in_price,out_weight_mean,out_value_mean,in_weight_mean,in_value_mean,tradebalance_mean
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,9,4,5,...,NaN,NaN,NaN,NaN,NaN,7207.5,33.75,4665.5,14.5,19.25
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,9,5,6,...,NaN,NaN,NaN,NaN,NaN,7207.5,33.75,4665.5,14.5,19.25
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,10,6,0,...,NaN,NaN,NaN,NaN,NaN,7207.5,33.75,4665.5,14.5,19.25
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,10,7,1,...,NaN,NaN,NaN,NaN,NaN,7207.5,33.75,4665.5,14.5,19.25
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,10,8,2,...,NaN,NaN,NaN,NaN,NaN,7207.5,33.75,4665.5,14.5,19.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J,2023,3,13,27,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1088,RD_F_J_20230328,2023-03-28,RD,F,J,2023,3,13,28,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1089,RD_F_J_20230329,2023-03-29,RD,F,J,2023,3,13,29,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1090,RD_F_J_20230330,2023-03-30,RD,F,J,2023,3,13,30,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
trade_month_mean = trade_df.groupby(['item', 'month'])[['out_weight', 'out_value', 'in_weight', 'in_value', 'tradebalance']].mean().reset_index()
cols = ['out_weight', 'out_value', 'in_weight', 'in_value', 'tradebalance']
items = ['BC', 'CB', 'CR', 'TG']
for item in items:
    for col in cols:
        mask_1 = train_df['item'] == item
        mask_2 = (trade_month_mean['item'] == item) & (trade_month_mean['month'] == 3)
        train_df.loc[mask_1, col] = train_df.loc[mask_1, col].fillna(trade_month_mean.loc[mask_2, col].values[0])
        test_df.loc[mask_1, col] = test_df.loc[mask_1, col].fillna(trade_month_mean.loc[mask_2, col].values[0])

train_df

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,in_weight,in_value,tradebalance,out_price,in_price,out_weight_mean,out_value_mean,in_weight_mean,in_value_mean,tradebalance_mean
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,0.0,0.0,172.0,0.002947,NaN,192621.6,317.4,0.2,0.0,317.4
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,0.0,0.0,172.0,0.002947,NaN,192621.6,317.4,0.2,0.0,317.4
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,0.0,0.0,172.0,0.002947,NaN,192621.6,317.4,0.2,0.0,317.4
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,0.0,0.0,172.0,0.002947,NaN,192621.6,317.4,0.2,0.0,317.4
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,0.0,0.0,172.0,0.002947,NaN,192621.6,317.4,0.2,0.0,317.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,9,...,2.0,0.0,4.0,0.001000,0.0,4000.0,4.0,2.0,0.0,4.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,9,...,2.0,0.0,4.0,0.001000,0.0,4000.0,4.0,2.0,0.0,4.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_df

,ID,timestamp,item,corporation,location,year,month,week,day,day_of_week,...,in_weight,in_value,tradebalance,out_price,in_price,out_weight_mean,out_value_mean,in_weight_mean,in_value_mean,tradebalance_mean
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,9,4,5,...,4665.5,14.5,19.25,NaN,NaN,7207.5,33.75,4665.5,14.5,19.25
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,9,5,6,...,4665.5,14.5,19.25,NaN,NaN,7207.5,33.75,4665.5,14.5,19.25
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,10,6,0,...,4665.5,14.5,19.25,NaN,NaN,7207.5,33.75,4665.5,14.5,19.25
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,10,7,1,...,4665.5,14.5,19.25,NaN,NaN,7207.5,33.75,4665.5,14.5,19.25
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,10,8,2,...,4665.5,14.5,19.25,NaN,NaN,7207.5,33.75,4665.5,14.5,19.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J,2023,3,13,27,0,...,4665.5,14.5,19.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1088,RD_F_J_20230328,2023-03-28,RD,F,J,2023,3,13,28,1,...,4665.5,14.5,19.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1089,RD_F_J_20230329,2023-03-29,RD,F,J,2023,3,13,29,2,...,4665.5,14.5,19.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1090,RD_F_J_20230330,2023-03-30,RD,F,J,2023,3,13,30,3,...,4665.5,14.5,19.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_df['out_price'] = test_df['out_value'] / test_df['out_weight']
test_df['in_price'] = test_df['in_value'] / test_df['in_weight']
test_df

,ID,timestamp,item,corporation,location,year,month,week,day,day_of_week,...,in_weight,in_value,tradebalance,out_price,in_price,out_weight_mean,out_value_mean,in_weight_mean,in_value_mean,tradebalance_mean
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,9,4,5,...,4665.5,14.5,19.25,0.004683,0.003108,7207.5,33.75,4665.5,14.5,19.25
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,9,5,6,...,4665.5,14.5,19.25,0.004683,0.003108,7207.5,33.75,4665.5,14.5,19.25
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,10,6,0,...,4665.5,14.5,19.25,0.004683,0.003108,7207.5,33.75,4665.5,14.5,19.25
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,10,7,1,...,4665.5,14.5,19.25,0.004683,0.003108,7207.5,33.75,4665.5,14.5,19.25
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,10,8,2,...,4665.5,14.5,19.25,0.004683,0.003108,7207.5,33.75,4665.5,14.5,19.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,RD,F,J,2023,3,13,27,0,...,4665.5,14.5,19.25,0.004683,0.003108,NaN,NaN,NaN,NaN,NaN
1088,RD_F_J_20230328,2023-03-28,RD,F,J,2023,3,13,28,1,...,4665.5,14.5,19.25,0.004683,0.003108,NaN,NaN,NaN,NaN,NaN
1089,RD_F_J_20230329,2023-03-29,RD,F,J,2023,3,13,29,2,...,4665.5,14.5,19.25,0.004683,0.003108,NaN,NaN,NaN,NaN,NaN
1090,RD_F_J_20230330,2023-03-30,RD,F,J,2023,3,13,30,3,...,4665.5,14.5,19.25,0.004683,0.003108,NaN,NaN,NaN,NaN,NaN


In [ ]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

train_df.isnull().sum().sum() , test_df.isnull().sum().sum()

(0, 0)

In [ ]:
# 품목별 평균 가격
item_month_mean_price = train_df.groupby(['item', 'month'])['price'].mean().reset_index()
item_month_mean_price = item_month_mean_price.rename(columns={'price' : 'item_month_mean_price'})
train_df = train_df.merge(item_month_mean_price, on=['item', 'month'], how='left')
test_df = test_df.merge(item_month_mean_price, on=['item', 'month'], how='left')

item_week_mean_price = train_df.groupby(['item', 'week'])['price'].mean().reset_index()
item_week_mean_price = item_week_mean_price.rename(columns={'price' : 'item_week_mean_price'})
train_df = train_df.merge(item_week_mean_price, on=['item', 'week'], how='left')
test_df = test_df.merge(item_week_mean_price, on=['item', 'week'], how='left')

train_df

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,tradebalance,out_price,in_price,out_weight_mean,out_value_mean,in_weight_mean,in_value_mean,tradebalance_mean,item_month_mean_price,item_week_mean_price
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,...,172.0,0.002947,0.0,192621.6,317.4,0.2,0.0,317.4,1588.269032,1302.388235
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,1,...,172.0,0.002947,0.0,192621.6,317.4,0.2,0.0,317.4,1588.269032,1302.388235
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,1,...,172.0,0.002947,0.0,192621.6,317.4,0.2,0.0,317.4,1588.269032,1302.388235
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,1,...,172.0,0.002947,0.0,192621.6,317.4,0.2,0.0,317.4,1588.269032,1302.388235
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,1,...,172.0,0.002947,0.0,192621.6,317.4,0.2,0.0,317.4,1588.269032,1302.388235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,9,...,4.0,0.001000,0.0,4000.0,4.0,2.0,0.0,4.0,291.963652,319.829545
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,9,...,4.0,0.001000,0.0,4000.0,4.0,2.0,0.0,4.0,291.963652,319.829545
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,9,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,275.096457,319.829545
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,9,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,275.096457,319.829545


In [ ]:
from sklearn.preprocessing import LabelEncoder

cols = ['item']

for col in cols:
    enc = LabelEncoder()
    train_df[col]=enc.fit_transform(train_df[col])
    test_df[col]=enc.transform(test_df[col])

train_df.head()

,ID,timestamp,item,corporation,location,supply,price,year,month,week,...,tradebalance,out_price,in_price,out_weight_mean,out_value_mean,in_weight_mean,in_value_mean,tradebalance_mean,item_month_mean_price,item_week_mean_price
0,TG_A_J_20190101,2019-01-01,4,A,J,0.0,0.0,2019,1,1,...,172.0,0.002947,0.0,192621.6,317.4,0.2,0.0,317.4,1588.269032,1302.388235
1,TG_A_J_20190102,2019-01-02,4,A,J,0.0,0.0,2019,1,1,...,172.0,0.002947,0.0,192621.6,317.4,0.2,0.0,317.4,1588.269032,1302.388235
2,TG_A_J_20190103,2019-01-03,4,A,J,60601.0,1728.0,2019,1,1,...,172.0,0.002947,0.0,192621.6,317.4,0.2,0.0,317.4,1588.269032,1302.388235
3,TG_A_J_20190104,2019-01-04,4,A,J,25000.0,1408.0,2019,1,1,...,172.0,0.002947,0.0,192621.6,317.4,0.2,0.0,317.4,1588.269032,1302.388235
4,TG_A_J_20190105,2019-01-05,4,A,J,32352.0,1250.0,2019,1,1,...,172.0,0.002947,0.0,192621.6,317.4,0.2,0.0,317.4,1588.269032,1302.388235


In [ ]:
train_df['item_id'] = train_df.ID.str[0:6]
test_df['item_id'] = test_df.ID.str[0:6]

In [ ]:
train_df['item_id'].unique()

array(['TG_A_J', 'TG_A_S', 'TG_B_J', 'TG_B_S', 'TG_C_J', 'TG_C_S',
       'TG_D_J', 'TG_D_S', 'TG_E_J', 'TG_E_S', 'CR_A_J', 'CR_B_J',
       'CR_C_J', 'CR_D_J', 'CR_D_S', 'CR_E_J', 'CR_E_S', 'CB_A_J',
       'CB_A_S', 'CB_D_J', 'CB_E_J', 'RD_A_J', 'RD_A_S', 'RD_C_S',
       'RD_D_J', 'RD_D_S', 'RD_E_J', 'RD_E_S', 'BC_A_J', 'BC_A_S',
       'BC_B_J', 'BC_B_S', 'BC_C_J', 'BC_C_S', 'BC_D_J', 'BC_E_J',
       'BC_E_S', 'CB_F_J', 'RD_F_J'], dtype=object)

In [ ]:
train_df.columns

Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply', 'price',
       'year', 'month', 'week', 'day', 'day_of_week', 'sin_month', 'cos_month',
       'sin_week', 'cos_week', 'sin_dayofweek', 'cos_dayofweek', 'harvest',
       'sunday', 'month_dayofweek_mean', 'month_dayofweek_std',
       'month_dayofweek_max', 'week_mean', 'week_std', 'month_mean',
       'month_std', 'month_mean_supply', 'month_std_supply',
       'week_mean_supply', 'week_std_supply', 'total', 'week_mean_total',
       'week_std_total', 'month_mean_total', 'month_std_total', 'out_weight',
       'out_value', 'in_weight', 'in_value', 'tradebalance', 'out_price',
       'in_price', 'out_weight_mean', 'out_value_mean', 'in_weight_mean',
       'in_value_mean', 'tradebalance_mean', 'item_month_mean_price',
       'item_week_mean_price', 'item_id'],
      dtype='object')

In [ ]:
cols = ["price", "supply", "total", "month", "week", 'year', 'out_weight', 'out_value', 'in_weight', 'in_value', 'tradebalance']
train_df[cols].corr()

,price,supply,total,month,week,year,out_weight,out_value,in_weight,in_value,tradebalance
price,1.000000,-0.002297,0.170748,-0.076673,-0.080900,0.029442,-0.057912,-0.013882,-0.223348,-0.226958,0.198818
supply,-0.002297,1.000000,0.767383,-0.120580,-0.122755,0.024278,0.046334,0.067230,-0.075297,-0.095893,0.106979
total,0.170748,0.767383,1.000000,-0.132087,-0.136061,0.053027,0.112890,0.186526,-0.106880,-0.136122,0.180492
month,-0.076673,-0.120580,-0.132087,1.000000,0.968650,-0.114541,0.186519,0.228641,0.057475,0.037315,0.038461
week,-0.080900,-0.122755,-0.136061,0.968650,1.000000,-0.110539,0.185063,0.227776,0.055034,0.034692,0.040538
year,0.029442,0.024278,0.053027,-0.114541,-0.110539,1.000000,-0.080826,-0.047561,-0.046471,-0.011065,-0.005049
out_weight,-0.057912,0.046334,0.112890,0.186519,0.185063,-0.080826,1.000000,0.933423,-0.127880,-0.161964,0.438404
out_value,-0.013882,0.067230,0.186526,0.228641,0.227776,-0.047561,0.933423,1.000000,-0.143568,-0.177159,0.472936
in_weight,-0.223348,-0.075297,-0.106880,0.057475,0.055034,-0.046471,-0.127880,-0.143568,1.000000,0.945050,-0.891198
in_value,-0.226958,-0.095893,-0.136122,0.037315,0.034692,-0.011065,-0.161964,-0.177159,0.945050,1.000000,-0.950945


4 == TG

2 == CR

1 == CB

3 == RD

0 == BC

In [ ]:
train_dropcols = ['ID', 'supply', 'total', 'timestamp', 'week_mean_supply', 'week_std_supply', 'week_mean_total', 'week_std_total', 'item_id', 'item', 'month_dayofweek_std', 'month_dayofweek_max', 'week_std', 'month_std', 'month_std_supply', 'month_std_total', 'month_mean_total']
test_dropcols = ['ID', 'timestamp', 'week_mean_supply', 'week_std_supply', 'week_mean_total', 'week_std_total', 'item_id', 'item', 'month_dayofweek_std', 'month_dayofweek_max', 'week_std', 'month_std', 'month_std_supply', 'month_std_total', 'month_mean_total']

train_ft = train_df[train_df.item == 4].drop(columns = train_dropcols)
test_ft = test_df[test_df.item == 4].drop(columns = test_dropcols)

target = train_df[train_df.item == 4]['price']

train_ft.shape, test_ft.shape, target.shape

((15230, 34), (280, 33), (15230,))

In [ ]:
train_ft.columns

Index(['corporation', 'location', 'price', 'year', 'month', 'week', 'day',
       'day_of_week', 'sin_month', 'cos_month', 'sin_week', 'cos_week',
       'sin_dayofweek', 'cos_dayofweek', 'harvest', 'sunday',
       'month_dayofweek_mean', 'week_mean', 'month_mean', 'month_mean_supply',
       'out_weight', 'out_value', 'in_weight', 'in_value', 'tradebalance',
       'out_price', 'in_price', 'out_weight_mean', 'out_value_mean',
       'in_weight_mean', 'in_value_mean', 'tradebalance_mean',
       'item_month_mean_price', 'item_week_mean_price'],
      dtype='object')

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

train = TabularDataset(train_ft)
test = TabularDataset(test_ft)

In [ ]:
predictor = TabularPredictor(label='price', eval_metric='rmse')
predictor.fit(train, presets='best_quality', num_gpus=1, hyperparameter_tune_kwargs = {
                                                                            'num_trials': 20,
                                                                            'scheduler' : 'local',
                                                                            'searcher': 'auto',
                                                                        })

Fitted model: NeuralNetTorch_BAG_L2/31fe9d45 ...
	-1513.3149	 = Validation score   (-root_mean_squared_error)
	153.47s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L2/a4447897 ...
	-1491.986	 = Validation score   (-root_mean_squared_error)
	286.35s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L2/e3292e35 ...
	-1497.4218	 = Validation score   (-root_mean_squared_error)
	250.3s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L2/3c1cee42 ...
	-1503.8252	 = Validation score   (-root_mean_squared_error)
	244.35s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: NeuralNetTorch_BAG_L2/515094f2 ...
	-1532.3863	 = Validation score   (-root_mean_squared_error)
	134.9s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBMLarge_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	-1440.2231	 = Validation score 

In [ ]:
ld_board = predictor.leaderboard(train, silent=False)
ld_board

                              model   score_test    score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0             KNeighborsDist_BAG_L1    -0.490332 -1666.332304        3.531156       0.000149     4.493423                 3.531156                0.000149           4.493423            1       True          2
1              ExtraTreesMSE_BAG_L1  -570.752054 -1547.176808        1.928663       0.000239    26.455218                 1.928663                0.000239          26.455218            1       True         19
2            RandomForestMSE_BAG_L1  -571.588011 -1550.617231        2.049686       0.000234    44.890467                 2.049686                0.000234          44.890467            1       True         13
3                 XGBoost_BAG_L1/T5  -939.946647 -1548.677857        1.229054       0.000195    48.678451                 1.229054                0.000195          

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,-0.490332,-1666.332304,3.531156,0.000149,4.493423,3.531156,0.000149,4.493423,1,True,2
1,ExtraTreesMSE_BAG_L1,-570.752054,-1547.176808,1.928663,0.000239,26.455218,1.928663,0.000239,26.455218,1,True,19
2,RandomForestMSE_BAG_L1,-571.588011,-1550.617231,2.049686,0.000234,44.890467,2.049686,0.000234,44.890467,1,True,13
3,XGBoost_BAG_L1/T5,-939.946647,-1548.677857,1.229054,0.000195,48.678451,1.229054,0.000195,48.678451,1,True,29
4,NeuralNetTorch_BAG_L2/e3292e35,-1051.448878,-1497.421820,167.838668,0.350996,6116.394193,0.901305,0.002469,250.304229,2,True,61
...,...,...,...,...,...,...,...,...,...,...,...,...
60,NeuralNetTorch_BAG_L1/f786f017,-1589.827835,-1614.353727,0.406187,0.000176,279.220535,0.406187,0.000176,279.220535,1,True,31
61,NeuralNetFastAI_BAG_L1/46790d0c,-1591.731039,-1604.506358,1.295781,0.000204,78.409791,1.295781,0.000204,78.409791,1,True,22
62,NeuralNetFastAI_BAG_L1/4696d46c,-1611.901182,-1621.046976,3.057986,0.000171,200.025515,3.057986,0.000171,200.025515,1,True,23
63,NeuralNetTorch_BAG_L1/7f4142c0,-1775.165519,-1848.376633,0.881867,0.000333,228.279037,0.881867,0.000333,228.279037,1,True,34


In [ ]:
pred_y = predictor.predict(test)

In [ ]:
pred_y

0      3242.980713
1        -0.401522
2      2996.892334
3      2906.065430
4      3197.655762
          ...     
275    4160.319824
276    4369.498047
277    4386.510742
278    4268.041992
279    4418.673828
Name: price, Length: 280, dtype: float32

In [ ]:
# predictor.feature_importance(train)
model_to_use = predictor.get_model_best()
pred = predictor.predict(test, model=model_to_use)
pred

0      3242.980713
1        -0.401522
2      2996.892334
3      2906.065430
4      3197.655762
          ...     
275    4160.319824
276    4369.498047
277    4386.510742
278    4268.041992
279    4418.673828
Name: price, Length: 280, dtype: float32

In [ ]:
submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
submission['item'] = submission.ID.str[0:2]
submission.loc[submission.item == 'TG', 'item'] = 4
submission.loc[submission.item == 'CR', 'item'] = 2
submission.loc[submission.item == 'CB', 'item'] = 1
submission.loc[submission.item == 'RD', 'item'] = 3
submission.loc[submission.item == 'BC', 'item'] = 0
submission.loc[submission.item == 4, 'answer'] = pred
submission

,ID,answer,item
0,TG_A_J_20230304,3242.980713,4
1,TG_A_J_20230305,-0.401522,4
2,TG_A_J_20230306,2996.892334,4
3,TG_A_J_20230307,2906.065430,4
4,TG_A_J_20230308,3197.655762,4
...,...,...,...
1087,RD_F_J_20230327,0.000000,3
1088,RD_F_J_20230328,0.000000,3
1089,RD_F_J_20230329,0.000000,3
1090,RD_F_J_20230330,0.000000,3


In [ ]:
submission.to_csv(f"{DATA_PATH}submission_autogluon_ver6.csv", index=False)